In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [3]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect, select, join, text

In [4]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [5]:
# reflect an existing database into a new model
Base = automap_base()

# reflect the tables
Base.prepare(engine, reflect=True)

In [6]:
# View all of the classes that automap found
Base.classes.keys()

['measurement', 'station']

In [7]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [8]:
# Create our session (link) from Python to the DB
session = Session(engine)

In [9]:
# Explore Database
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [10]:
# Get a list of column names and types for Station
columns = inspector.get_columns('station')

for c in columns:
    print(c['name'], c["type"])


id INTEGER
station TEXT
name TEXT
latitude FLOAT
longitude FLOAT
elevation FLOAT


In [11]:
# Get a list of column names and types for Measurement
columns = inspector.get_columns('measurement')

for c in columns:
    print(c['name'], c["type"])

id INTEGER
station TEXT
date TEXT
prcp FLOAT
tobs FLOAT


# Exploratory Precipitation Analysis

In [ ]:
# View Measurment Data
engine.execute('SELECT * FROM Measurement LIMIT 5').fetchall()

In [ ]:
# View Station Data
engine.execute('SELECT * FROM Station LIMIT 5').fetchall()

In [ ]:
# Find the most recent date in the data set.
recent = session.query(Measurement.date).order_by(Measurement.date.desc()).first()
print(recent)

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 

# Starting from the most recent data point in the database. 

# Calculate the date one year from the last date in data set.
one_year = dt.date(2017, 8, 23) - dt.timedelta(days=365)

# Perform a query to retrieve the data and precipitation scores
precipitation_data = session.query(Measurement.date, Measurement.prcp).\
    filter(Measurement.date > one_year).\
    order_by(Measurement.date).all()

# Save the query results as a Pandas DataFrame and set the index to the date column
precipitation_df = pd.DataFrame(precipitation_data)
precipitation_df.set_index('date', inplace=True)

# Sort the dataframe by date
precipitation_df_sorted = precipitation_df.sort_values("date", ascending=True)

# Use Pandas Plotting with Matplotlib to plot the data
axis = precipitation_df_sorted.plot(figsize=(10,5))
axis.set_title("Hawaii Precipitation from 8-24-16 to 8-23-17")
plt.show()

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
mean = precipitation_df_sorted['prcp'].mean()
median = precipitation_df_sorted['prcp'].median()
mode = precipitation_df_sorted['prcp'].mode()

mean_formatted = "{:.2f}".format(mean)
print(f'The mean is {mean_formatted}, the median is {median}, and the mode is {mode}')

In [ ]:
# Determine if there are any potential outliers
quartiles = precipitation_df_sorted['prcp'].quantile([.25,.5,.75])
lowerq = quartiles[0.25]
upperq = quartiles[0.75]
iqr = upperq-lowerq
print(f"The lower quartile of precipitation is: {lowerq}")
print(f"The upper quartile of precipitation is: {upperq}")
print(f"The interquartile range of precipitation is: {iqr}")
print(f"The median of precipitation is: {quartiles[0.5]} ")
lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

In [ ]:
# Use describe to view summary stats of dataframe
precipitation_df_sorted.describe()

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
total_stations = session.query(Station.station).count()
print(f'There are a total of {total_stations} stations in the dataset')

In [ ]:
# Design a query to find the most active stations (i.e. which stations have the most rows?).
active_stations = session.query(Measurement.station,func.count(Measurement.station).label('count')).\
    group_by(Measurement.station).order_by(text('count DESC')).all()

# List the stations and observation counts in descending order.
active_stations

In [41]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
temp_max = session.query(func.max(Measurement.tobs)).filter(Measurement.station == 'USC00519281').all()
temp_min = session.query(func.min(Measurement.tobs)).filter(Measurement.station == 'USC00519281').all()
temp_avg = session.query(func.avg(Measurement.tobs)).filter(Measurement.station == 'USC00519281').all()

print(f"The max temp is {[list[0] for list in temp_max]}, \
        the min temp is {[list[0] for list in temp_min]}, \
        and the average temp is {[list[0] for list in temp_avg]} \
        for station USC00519281")

The max temp is [85.0],         the min temp is [54.0],         and the average temp is [71.66378066378067]         for station USC00519281


In [ ]:
# Using the most active station id 
# Query the last 12 months of temperature observation data for this station 

most_active_station_data = session.query(Measurement.date, Measurement.tobs).\
     filter(Measurement.date > one_year).\
     filter(Measurement.station == 'USC00519281').all()

# Save the query results as a Pandas DataFrame and set the index to the date column
most_active_df = pd.DataFrame(most_active_station_data)
most_active_df.set_index('date', inplace=True)
most_active_df

In [ ]:
# Plot the results as a histogram
plt.hist(most_active_df, bins=12)
plt.title("TOBS for WAIHEE")
plt.xlabel("Temperature")
plt.ylabel("Frequency")
plt.show()

# Close session

In [15]:
# Close Session
session.close()